## BO active learning with GP - Tuning sample height on the instrument
#### Contributor(s): Utkarsh Pratiush <utkarshp1161@gmail.com> - 18th April 2025
#### edited - 
   

## Steup microscope

In [18]:
from stemOrchestrator.acquisition import TFacquisition
from stemOrchestrator.process import HAADF_tiff_to_png
from autoscript_tem_microscope_client import TemMicroscopeClient
from stemOrchestrator.logging_config   import setup_logging
import matplotlib.pyplot as plt
plot = plt

from typing import Dict

In [19]:
import os
import json
from pathlib import Path

ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")

if not ip or not port:
    secret_path = Path("../../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF_wd")
            port = port or secret.get("port_TF_wd")


if not ip:
    ip = input("Enter microscope IP: ")
if not port:
    port = input("Enter microscope Port: ")

try:
    port = int(port)
except ValueError:
    raise ValueError("Port must be an integer")

config = {
    "ip": ip,
    "port": port,
    "haadf_exposure": 40e-8,  # micro-seconds per pixel
    "haadf_resolution": 512, # square
    "out_path": "."
}

In [20]:
data_folder  = "../../../data/"
out_path = data_folder 

In [21]:

ip = config["ip"]
port = config["port"]
haadf_exposure = config["haadf_exposure"]
out_path = config["out_path"]
haadf_resolution = config["haadf_resolution"]


setup_logging(out_path=out_path)

microscope = TemMicroscopeClient()
microscope.connect(ip, port = port)# 7521 on velox  computer

# query state:

tf_acquisition = TFacquisition(microscope=microscope)

# put beam shift to 0,0
# tf_acquisition.move_beam_shift_positon([0, 0])


Client connecting to [10.1.149.210:9090]...
Client connected to [10.1.149.210:9090]
2025-06-20 10:08:24,086 - INFO - Starting microscope initialization...


2025-06-20 10:08:24,648 - INFO - CETA camera initialized
2025-06-20 10:08:24,654 - INFO - HAADF detector initialized
2025-06-20 10:08:24,661 - INFO - Microscope initialization completed successfully


## Setup tuning -> sample-height


In [22]:
import numpy as np
from stemOrchestrator.MLlayer.MLlayerBO import Tune1d
import torch


contrast_value = []

def z_height(x_old: float, y_old: float, z: int):
    # mic 
    #z_lim: Limits(min=-0.000375, max=0.000375)
    # lets get a HAADF image

    print(x_old, y_old,z/1e6)
    tf_acquisition.move_stage_translation_absolute(x=x_old, y=y_old,z=z/1e6 )

    # get image 1 for drift
    image_data, haadf_tiff_name, pixel_size_tuple = tf_acquisition.acquire_haadf(exposure = 1e-5, resolution=128, return_pixel_size=True )
    normalized_image_data = image_data - np.min(image_data)
    normalized_image_data = normalized_image_data / np.max(normalized_image_data)
    gerd_contrast = np.std(normalized_image_data) / np.average(normalized_image_data)
    contrast_value.append(gerd_contrast)

    # For simplicity, choose one of the contrast measures as output `y`
    y = gerd_contrast

    HAADF_tiff_to_png(haadf_tiff_name)
    print(f"Gerd Contrast{gerd_contrast}")

    np.save(f"at_z=_{z}with_y={y}",normalized_image_data)

    return y


In [10]:
# Define the optimization problem
variable = 'x'
python_command =  z_height
num_gp_steps = 20
bounds_in_micrometer = torch.tensor([[-120.0], [120.0]])  # Shape is [2, 1]--> range in which search for optimal value




In [11]:
pos = tf_acquisition.query_stage_position()
old_x, old_y, _, _ = pos[0], pos[1], pos[2], pos[3]

2025-06-20 10:03:37,874 - INFO - Querying the stage position now: the stage position (X, Y, Z) in meters, A and B in radians
2025-06-20 10:03:37,875 - INFO - Query stage position: which is at StagePosition(x=5.9888376e-05, y=6.486134e-05, z=0, a=0, b=0), i.e pos in nm {'x': 5.988837587639355e-14, 'y': 6.486133987807655e-14, 'z': 0.0}, angles in mili degree0.0


In [16]:
tf_acquisition.query_stage_position()

2025-06-20 10:07:06,134 - INFO - Querying the stage position now: the stage position (X, Y, Z) in meters, A and B in radians
2025-06-20 10:07:06,136 - INFO - Query stage position: which is at StagePosition(x=5.993628e-05, y=6.4913236e-05, z=0, a=0, b=0), i.e pos in nm {'x': 5.993627987639149e-14, 'y': 6.491323587807206e-14, 'z': 0.0}, angles in mili degree0.0


StagePosition(x=5.993628e-05, y=6.4913236e-05, z=0, a=0, b=0)

In [12]:
# Create and run the optimizer
# default is maximization so ----> put a negative sign to minimize

optimizer = Tune1d(variable, python_command, old_x, old_y, num_gp_steps, bounds_in_micrometer, seed_pt=42)
best_x, best_y = optimizer.optimize()

print(f"Best x: {best_x}")
print(f"Best y (std dev): {best_y}")
tf_acquisition.move_stage_translation_absolute(x=old_x, y=old_y, z=best_x/1e6 )


5.988837587639355e-05 6.486133987807654e-05 9.174462890625e-05
2025-06-20 10:03:42,484 - INFO - Request to translate the stage absolute values


ApplicationServerException: Stage failed to move to desired position.
Stage not ready after 500.0000 ms.